In [19]:
# import findspark
import findspark
# initialize findspark with spark directory
findspark.init("C:\BigData\BigData\spark-3.1.2-bin-hadoop3.2")
# import pyspark
import pyspark
# create spark context
sc = pyspark.SparkContext()
# create spark session 
spark = pyspark.sql.SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at C:\Users\vikto\AppData\Local\Temp\ipykernel_20996\2064577265.py:8 

In [20]:
# check
spark

In [67]:
import emojis 

In [97]:
!pip3 install sparknlp

In [98]:
import sparknlp

In [94]:
# import packages
import os 
import pickle

import re
from datetime import datetime

import requests

import pytz
import emojis

import pandas as pd
import numpy as np

import ast

import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover

In [163]:
import tweepy
import csv
import time
import pandas as pd
import datetime
import os
import json
from pandas.tseries.holiday import nearest_workday, \
    AbstractHolidayCalendar, Holiday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, \
    USMemorialDay, USLaborDay, USThanksgivingDay
from datetime import date

# 1. Import Data & Exploration

In [112]:
list_brands = ["healthyfood",
               "healthylifestyle",
               "_vegan_",
               "keto",
               "ketodiet",
               "ketolifestyle",
               "veganism",
               "vegetarian"]
from re import search



data_dir = ".././../data/Topic_vegan/"
tweet_files = [os.path.join(data_dir, obs) for obs in os.listdir(data_dir)]



files_brand = [file for file in tweet_files if (file.find(list_brands[2]) != -1)]
files_brand               
               
df_json = spark.read.option("multiline","true").json(files_brand)  
df_json.count()

1595676

In [113]:
# check the schema of our json dataframe
df_json.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [114]:
# select interesting features
df = df_json.select(F.col("user.name"),
                                F.col("user.screen_name"),
                                F.col("created_at"), 
                                F.col("full_text"),
                                F.col("entities.hashtags"),
                                F.col("lang"),
                                F.col("favorite_count"),
                                F.col("retweet_count"),
                                F.col("user.followers_count"),
                                F.col("user.friends_count"),
                                F.col("user.favourites_count"),
                                F.col("entities.urls"),
                                F.col("entities.symbols"))

# 2. Preprocess Data

## 2.1 Check Time Period 

In [115]:
# https://developer.twitter.com/en/docs/twitter-ads-api/timezones
# function to convert Twitter date string format
def getDate(date):
    if date is not None:
        return str(datetime.strptime(date,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

# UDF declaration
date_udf = F.udf(getDate, StringType())

# apply udf
df = df.withColumn('post_created_at', F.to_utc_timestamp(date_udf("created_at"), "UTC"))

In [45]:
# show
df.select("created_at", "post_created_at").show(5, truncate=False)

+------------------------------+-------------------+
|created_at                    |post_created_at    |
+------------------------------+-------------------+
|Tue Sep 13 22:32:32 +0000 2022|2022-09-13 22:32:32|
|Tue Sep 13 22:32:26 +0000 2022|2022-09-13 22:32:26|
|Tue Sep 13 22:32:26 +0000 2022|2022-09-13 22:32:26|
|Tue Sep 13 22:32:16 +0000 2022|2022-09-13 22:32:16|
|Tue Sep 13 22:32:09 +0000 2022|2022-09-13 22:32:09|
+------------------------------+-------------------+
only showing top 5 rows



In [116]:
#drop duplicates and retweets 
df = df.filter(~F.col("full_text").startswith("RT"))\
                        .drop_duplicates()
#sorting such when dropping later we only keep the most recent post 
df = df.sort("post_created_at", ascending=False)
#removing spam accounts 
df = df.drop_duplicates(["full_text", "screen_name"])

#df.printSchema()
#df.count() #1340938

# Feature Engineering

In [117]:
# define function to count hashtags
def get_hashtags(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "#" in word:
            counter += 1
    return(counter)

In [118]:
# define function to count mentions
def get_mentions(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "@" in word:
            counter += 1
    return(counter)

In [119]:
# define function to count exclamation marks
def get_exclamation_marks(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "!" in word:
            counter += 1
    return(counter)

In [120]:
# define function to count number of emojis used
def emoji_counter(text):
    nr_emojis = emojis.count(text)
    return(nr_emojis)

In [121]:
# define function to calculate engagement rate
def engagement_rate(favorite_count, retweet_count, followers_count):
    if(followers_count == 0):
        eng_rate = 0
    else:
        eng_rate = (favorite_count + retweet_count)/followers_count
    
    return eng_rate

In [122]:
# register functions as udf
get_hashtags_UDF = F.udf(get_hashtags, IntegerType())
get_mentions_UDF = F.udf(get_mentions, IntegerType())
get_exclamation_marks_UDF = F.udf(get_exclamation_marks, IntegerType())
emoji_counter_UDF = F.udf(emoji_counter, IntegerType())
engagement_rate_UDF = F.udf(engagement_rate, DoubleType())

In [123]:
# apply functions to create new features
df = df.withColumn("emoji_count", emoji_counter_UDF("full_text")) \
        .withColumn("text_tokenized", F.split("full_text", " ")) \
        .withColumn("num_words", F.size("text_tokenized")) \
        .withColumn("num_hashtags", get_hashtags_UDF("text_tokenized")) \
        .withColumn("num_mentions", get_mentions_UDF("text_tokenized")) \
        .withColumn("num_exclamation_marks", get_exclamation_marks_UDF("text_tokenized")) \
        .withColumn("engagement_rate", engagement_rate_UDF("favorite_count", "retweet_count", "followers_count"))

# show
df.select("full_text", "emoji_count", "num_words", "num_hashtags", "num_mentions", "num_exclamation_marks", "engagement_rate").toPandas().tail(5)

full_text  \
592246                            Check out Maddie Payton's video! #TikTok https://t.co/g46yyGy77U #growth #growthmindset #believe #retweet #socialmedia #marketing #contentcreator #influencer #wcw #vegan #tbt #disney #Disneytiktok #affiliatemarketing #trending #trends #fyp #marketing #lifestyle #youtube #hashtag   
592247                                                                                                                                                                                                                                                                                  ????????? https://t.co/rIWy3nd1tP   
592248                                   @YourHostEdge I have sensory issues related to texture and taste that would make it exceedingly difficult to completely switch my diet.\n\nI respect other people’s decisions to go vegan but I don’t see why I should have to go thru it when it could literally cause me pain.   
592249                                                                                                                                                                                                                                    My Road To Being A Vegan | Alan Cox | TEDxLambethSalon\nhttps://t.co/VPfV8rnQjx   
592250  AD|PR - Looking for new beauty products? Read my favourite vegan beauty products\n👇Read More👇\n\nhttps://t.co/YcECxU1IcU #thegirlgang #lbloggers #blogginggals #GRLPWR @sotonbloggers #bloggerstribe #influencerrt #theclqrt #bloggershutrt #teacupclub #bloggerssparkle #cosybloggerclub https://t.co/dRfI2eVvdc   

        emoji_count  num_words  num_hashtags  num_mentions  \
592246            0         28            22             0   
592247            0          2             0             0   
592248            0         47             0             1   
592249            0         11             0             0   
592250            2         27            11             1   

        num_exclamation_marks  engagement_rate  
592246                      1         0.000000  
592247                      0         0.000000  
592248                      0         0.000000  
592249                      0         0.000000  
592250                      0         0.000091

# 3. Text Cleaning 

In [124]:
# filter for english tweets (NOTE: for the assignment you can translate non-english tweets using an API)
df = df.filter(F.col("lang") == "en")

In [85]:
# check number of observations
df.count()

439169

In [125]:
# define function to clean text
def clean_text(string):
    
    # define numbers
    NUMBERS = '0123456789'
    PUNCT = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    
    # convert text to lower case
    cleaned_string = string.lower()
    
    # remove URLS
    cleaned_string = re.sub(r'http\S+', ' ', cleaned_string)
    
    # replace emojis by words
    cleaned_string = emoji.demojize(cleaned_string)
    cleaned_string = cleaned_string.replace(":"," ").replace("_"," ")
    cleaned_string = ' '.join(cleaned_string.split())
    
    # remove numbers
    cleaned_string = "".join([char for char in cleaned_string if char not in NUMBERS])
    
    # remove punctuation
    cleaned_string = "".join([char for char in cleaned_string if char not in PUNCT])
    
    # remove words conisting of one character (or less)
    cleaned_string = ' '.join([w for w in cleaned_string.split() if len(w) > 1])
    
    # return
    return(cleaned_string)

In [126]:
# convert to udf
clean_text_udf = F.udf(clean_text, StringType())

In [127]:
# clean string
df = df.withColumn("cleaned_text", clean_text_udf(F.col("full_text")))

In [128]:
# check
pd.set_option('display.max_colwidth', None)
df.select("full_text", "cleaned_text").limit(5).toPandas()

full_text  \
0  "A lot of people say they can't eat vegan, but I'm just like them – and if I can do it they can do it." Nichole Lewis who is a 26-year-old mom lost 120 pounds and improved her #mentalhealth by eating a plant-based diet. Here are her weight loss secrets: https://t.co/4RAT8rwqTO   
1                                       "By now, your [vegan/vegetarian] lifestyle may be so easy to maintain that you don't even have to think about it...All of this can change when you study abroad." Check out Hannah's tips on staying vegan while abroad: https://t.co/fIy45ntvTD   
2         "Don't stay in the sun"\n"Use sunscreen"\n"Red meat is bad for you"\n"Vegan diet is healthy"\n"Seed oil is healthy"\n"Inflation is a good thing"\n"#Bitcoin is bad for the enviroment"\n\nWhat other things does the media portray as positive while being extremely damaging?   
3                                                             "Heroes sacrifice for causes; they do things that others hide from. I may not be some great hero, but I won't hide from this" - Brandon Mull.\n#Antispeciesism \n#Animalliberation \n#Vegan\n#Love https://t.co/1hK7FXsgRo   
4                                                                                           "Last year over a million people left the same suicide note... SHOPPING LIST: Butter, Eggs, Milk, Cheese, Beef, Chicken, and Bacon." - Physicians Committee for Responsible Medicine\n#vegan   

                                                                                                                                                                                                                                         cleaned_text  
0                   lot of people say they cant eat vegan but im just like them and if can do it they can do it nichole lewis who is yearold mom lost pounds and improved her mentalhealth by eating plantbased diet here are her weight loss secrets  
1                                         by now your veganvegetarian lifestyle may be so easy to maintain that you dont even have to think about itall of this can change when you study abroad check out hannahs tips on staying vegan while abroad  
2  dont stay in the sun use sunscreen red meat is bad for you vegan diet is healthy seed oil is healthy inflation is good thing bitcoin is bad for the enviroment what other things does the media portray as positive while being extremely damaging  
3                                                                         heroes sacrifice for causes they do things that others hide from may not be some great hero but wont hide from this brandon mull antispeciesism animalliberation vegan love  
4                                                                           last year over million people left the same suicide note shopping list butter eggs milk cheese beef chicken and bacon physicians committee for responsible medicine vegan

Tokenizing the text 

In [129]:
#tokenize the cleaned_text variable 
tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="tokens")
df = tokenizer.transform(df)

In [130]:
df.select('cleaned_text', 'tokens').show()

+--------------------+--------------------+
|        cleaned_text|              tokens|
+--------------------+--------------------+
|lot of people say...|[lot, of, people,...|
|by now your vegan...|[by, now, your, v...|
|dont stay in the ...|[dont, stay, in, ...|
|heroes sacrifice ...|[heroes, sacrific...|
|last year over mi...|[last, year, over...|
|monday mint motiv...|[monday, mint, mo...|
|no nut november m...|[no, nut, novembe...|
|sir why did you s...|[sir, why, did, y...|
|there are many go...|[there, are, many...|
|vegan diets provi...|[vegan, diets, pr...|
|we were given veg...|[we, were, given,...|
|why are vegans so...|[why, are, vegans...|
|would you ever go...|[would, you, ever...|
|you doing anythin...|[you, doing, anyt...|
|defeating russia ...|[defeating, russi...|
|sustainabledecisi...|[sustainabledecis...|
|arfeatures cai st...|[arfeatures, cai,...|
|bgan punks are no...|[bgan, punks, are...|
|cbdoil for anxiet...|[cbdoil, for, anx...|
|didyouknow as lon...|[didyoukno

In [131]:
#remove stop words 
remover = StopWordsRemover(inputCol="tokens", outputCol="clean_tokens")
df = remover.transform(df)
df.select('tokens', 'clean_tokens').show()

+--------------------+--------------------+
|              tokens|        clean_tokens|
+--------------------+--------------------+
|[lot, of, people,...|[lot, people, say...|
|[by, now, your, v...|[veganvegetarian,...|
|[dont, stay, in, ...|[dont, stay, sun,...|
|[heroes, sacrific...|[heroes, sacrific...|
|[last, year, over...|[last, year, mill...|
|[monday, mint, mo...|[monday, mint, mo...|
|[no, nut, novembe...|[nut, november, m...|
|[sir, why, did, y...|[sir, start, laug...|
|[there, are, many...|[many, good, reas...|
|[vegan, diets, pr...|[vegan, diets, pr...|
|[we, were, given,...|[given, vegan, se...|
|[why, are, vegans...|[vegans, annoying...|
|[would, you, ever...|[ever, go, vegeta...|
|[you, doing, anyt...|[anything, specia...|
|[defeating, russi...|[defeating, russi...|
|[sustainabledecis...|[sustainabledecis...|
|[arfeatures, cai,...|[arfeatures, cai,...|
|[bgan, punks, are...|[bgan, punks, vegan]|
|[cbdoil, for, anx...|[cbdoil, anxiety,...|
|[didyouknow, as, ...|[didyoukno

In [132]:
# Stem text
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')
stemmer_udf = F.udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
df = df.withColumn("tokens_stemmed", stemmer_udf("clean_tokens"))

In [133]:
df.select('clean_tokens', 'tokens_stemmed').show()

+--------------------+--------------------+
|        clean_tokens|      tokens_stemmed|
+--------------------+--------------------+
|[lot, people, say...|[lot, peopl, say,...|
|[veganvegetarian,...|[veganvegetarian,...|
|[dont, stay, sun,...|[dont, stay, sun,...|
|[heroes, sacrific...|[hero, sacrific, ...|
|[last, year, mill...|[last, year, mill...|
|[monday, mint, mo...|[monday, mint, mo...|
|[nut, november, m...|[nut, novemb, mea...|
|[sir, start, laug...|[sir, start, laug...|
|[many, good, reas...|[mani, good, reas...|
|[vegan, diets, pr...|[vegan, diet, pro...|
|[given, vegan, se...|[given, vegan, se...|
|[vegans, annoying...|[vegan, annoy, an...|
|[ever, go, vegeta...|[ever, go, vegeta...|
|[anything, specia...|[anyth, special, ...|
|[defeating, russi...|[defeat, russia, ...|
|[sustainabledecis...|[sustainabledecis...|
|[arfeatures, cai,...|[arfeatur, cai, s...|
|[bgan, punks, vegan]| [bgan, punk, vegan]|
|[cbdoil, anxiety,...|[cbdoil, anxieti,...|
|[didyouknow, long...|[didyoukno

# 4. Sentiment 

## 4.1 The Vader Package

In [138]:
!pip3 install vaderSentiment

In [139]:
#using the vaderSentiment package 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [141]:
# define the function to extract the sentiment
def get_sentiment(sentence):

    # initialize sentiment analyzer
    sid_obj = SentimentIntensityAnalyzer()

    # get sentiment dict
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    # get positive sentiment score
    pos_sentiment = sentiment_dict["pos"]
    
    # return positive sentiment score
    return(pos_sentiment)

get_sentiment_udf = udf(get_sentiment, DoubleType())

In [152]:
df = df.withColumn("sentiment_vader", get_sentiment_udf(F.col("cleaned_text")))

In [153]:
df.select("cleaned_text", "sentiment_vader").show()

+--------------------+---------------+
|        cleaned_text|sentiment_vader|
+--------------------+---------------+
|lot of people say...|          0.139|
|by now your vegan...|          0.094|
|dont stay in the ...|            0.2|
|heroes sacrifice ...|          0.321|
|last year over mi...|          0.075|
|monday mint motiv...|          0.098|
|no nut november m...|          0.238|
|sir why did you s...|          0.263|
|there are many go...|          0.114|
|vegan diets provi...|          0.314|
|we were given veg...|           0.16|
|why are vegans so...|          0.164|
|would you ever go...|            0.0|
|you doing anythin...|          0.172|
|defeating russia ...|            0.0|
|sustainabledecisi...|          0.061|
|arfeatures cai st...|            0.0|
|bgan punks are no...|            0.0|
|cbdoil for anxiet...|          0.372|
|didyouknow as lon...|            0.0|
+--------------------+---------------+
only showing top 20 rows



## 4.2 TextBlob Package 

In [154]:
#use polarity and subjectivity from TextBlob 
#https://textblob.readthedocs.io/en/dev/
from textblob import TextBlob

In [155]:
# define function to get polarity score of text document
def get_polarity(row):
    textBlob_review = TextBlob(row)
    return textBlob_review.sentiment[0]
# define function to get subjectivity score of text document
def get_subjectivity(row):
    textBlob_review = TextBlob(row)
    return textBlob_review.sentiment[1]
get_polarity_udf = F.udf(get_polarity, DoubleType())
get_subjectivity_udf = F.udf(get_subjectivity, DoubleType())

In [156]:
df = df.withColumn('polarity', get_polarity_udf(F.col('cleaned_text')))\
        .withColumn('subjectivity', get_subjectivity_udf(F.col('cleaned_text')))

In [157]:
df.select("cleaned_text", "sentiment_vader", "polarity", "subjectivity").show()

+--------------------+---------------+--------------------+-------------------+
|        cleaned_text|sentiment_vader|            polarity|       subjectivity|
+--------------------+---------------+--------------------+-------------------+
|lot of people say...|          0.139|                 0.0|                0.0|
|by now your vegan...|          0.094| 0.43333333333333335| 0.8333333333333334|
|dont stay in the ...|            0.2|0.030808080808080826| 0.5393097643097643|
|heroes sacrifice ...|          0.321|                0.65|              0.675|
|last year over mi...|          0.075|-0.07999999999999999| 0.3383333333333333|
|monday mint motiv...|          0.098|                 0.0|                0.0|
|no nut november m...|          0.238|            0.221875|0.47187500000000004|
|sir why did you s...|          0.263|  0.6166666666666667| 0.4666666666666666|
|there are many go...|          0.114|  0.2785714285714286|               0.55|
|vegan diets provi...|          0.314|  

# 5. Basetable 

In [159]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- full_text: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- urls: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- display_url: string (nullable = true)
 |    |    |-- expanded_url: string (nullable = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- url: string (nullable = true)


## 5.1 Adjust dates to opening hours stock exchange

In [164]:
class USTradingHolidaysCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday(
            'NewYearsDay',
            month=1,
            day=1,
            observance=nearest_workday
        ),
        USMartinLutherKingJr,
        USPresidentsDay,
        GoodFriday,
        USMemorialDay,
        Holiday(
            'Juneteenth National Independence Day',
            month=6,
            day=19,
            start_date='2021-06-18',
            observance=nearest_workday,
        ),
        Holiday(
            'USIndependenceDay',
            month=7,
            day=4,
            observance=nearest_workday
        ),
        USLaborDay,
        USThanksgivingDay,
        Holiday(
            'Christmas',
            month=12,
            day=25,
            observance=nearest_workday
        ),
    ]
cal = USTradingHolidaysCalendar()
holidays = cal.holidays(start='2021-10-11', end='2022-10-12')

In [170]:
def opening_hours(tweet_date):
    tweet_date = pd.to_datetime(tweet_date)
    open_NYSE = tweet_date.replace(hour=14, minute=30, second=0, microsecond=0)
    close_NYSE = tweet_date.replace(hour=21, minute=0, second=0, microsecond=0)
    if (tweet_date < open_NYSE):
        dependent_date = tweet_date.date()
    else:
        dependent_date = tweet_date.date()
        if(dependent_date.isoweekday() == 5):
            dependent_date = dependent_date + datetime.timedelta(days = 3)
        elif(dependent_date.isoweekday() == 6):
            dependent_date = dependent_date + datetime.timedelta(days = 2)
        else:
            dependent_date = dependent_date + datetime.timedelta(days = 1)
    return dependent_date

In [172]:
opening_hours_udf = udf(opening_hours, DateType())
df = df.withColumn('date', opening_hours_udf(F.col('created_at')))
df.select('date', 'created_at').show()

+----------+--------------------+
|      date|          created_at|
+----------+--------------------+
|2021-11-03|Tue Nov 02 18:33:...|
|2022-03-11|Thu Mar 10 14:30:...|
|2022-04-27|Tue Apr 26 18:00:...|
|2022-06-21|Tue Jun 21 11:34:...|
|2022-05-02|Sat Apr 30 18:58:...|
|2021-12-14|Mon Dec 13 18:22:...|
|2021-11-03|Tue Nov 02 20:54:...|
|2022-05-02|Fri Apr 29 18:40:...|
|2022-01-12|Tue Jan 11 20:05:...|
|2022-04-27|Wed Apr 27 07:09:...|
|2022-08-01|Sun Jul 31 16:54:...|
|2022-08-11|Wed Aug 10 16:53:...|
|2022-08-03|Tue Aug 02 17:00:...|
|2021-11-02|Mon Nov 01 21:57:...|
|2022-03-09|Tue Mar 08 18:19:...|
|2022-03-31|Thu Mar 31 12:30:...|
|2022-06-29|Wed Jun 29 12:30:...|
|2021-12-31|Thu Dec 30 21:56:...|
|2022-03-14|Fri Mar 11 16:15:...|
|2022-05-17|Mon May 16 15:25:...|
+----------+--------------------+
only showing top 20 rows



## 5.2 Dependent Variable

In [173]:
stock_information = pd.read_excel("./../../data/stock_information.xlsx")

In [174]:
stock_information["close_yesterday"] = stock_information["close"].shift(1)
#stock_information = stock_information[stock_information.date != "2021-10-11"]
stock_information["dependent"] = (stock_information["close"] - stock_information["close_yesterday"])/stock_information["close_yesterday"]
display(stock_information)

symbol       date       open       high        low      close  volume  \
0     VEGN 2021-10-11  40.020000  40.161999  39.759998  39.759998    5000   
1     VEGN 2021-10-12  39.840000  39.840000  39.669998  39.724998    4900   
2     VEGN 2021-10-13  40.070000  40.099998  39.810001  39.924000   16000   
3     VEGN 2021-10-14  40.340000  40.765999  40.340000  40.765999    3100   
4     VEGN 2021-10-15  41.060001  41.150002  41.060001  41.122002    1700   
..     ...        ...        ...        ...        ...        ...     ...   
248   VEGN 2022-10-05  31.469999  31.628000  31.120001  31.628000    1800   
249   VEGN 2022-10-06  31.559999  31.600000  31.309999  31.309999    1200   
250   VEGN 2022-10-07  30.299999  30.299999  30.180000  30.239000    1300   
251   VEGN 2022-10-10  30.350000  30.350000  29.629999  29.830000    4500   
252   VEGN 2022-10-11  29.760000  29.860001  29.540001  29.540001    3200   

      adjusted  close_yesterday  dependent  
0    39.524410              NaN        NaN  
1    39.489620        39.759998  -0.000880  
2    39.687439        39.724998   0.005009  
3    40.524448        39.924000   0.021090  
4    40.878345        40.765999   0.008733  
..         ...              ...        ...  
248  31.628000        31.719999  -0.002900  
249  31.309999        31.628000  -0.010054  
250  30.239000        31.309999  -0.034206  
251  29.830000        30.239000  -0.013526  
252  29.540001        29.830000  -0.009722  

[253 rows x 10 columns]

In [175]:
def make_dependent(x,threshold):
    if x < -threshold:
        x = -1
    elif x > threshold:
        x = 1
    else:
        x = 0
    return x

In [177]:
stock_information["dependent"] = stock_information["dependent"].apply(lambda x: make_dependent(x,0.01))
display(stock_information)

symbol       date       open       high        low      close  volume  \
0     VEGN 2021-10-11  40.020000  40.161999  39.759998  39.759998    5000   
1     VEGN 2021-10-12  39.840000  39.840000  39.669998  39.724998    4900   
2     VEGN 2021-10-13  40.070000  40.099998  39.810001  39.924000   16000   
3     VEGN 2021-10-14  40.340000  40.765999  40.340000  40.765999    3100   
4     VEGN 2021-10-15  41.060001  41.150002  41.060001  41.122002    1700   
..     ...        ...        ...        ...        ...        ...     ...   
248   VEGN 2022-10-05  31.469999  31.628000  31.120001  31.628000    1800   
249   VEGN 2022-10-06  31.559999  31.600000  31.309999  31.309999    1200   
250   VEGN 2022-10-07  30.299999  30.299999  30.180000  30.239000    1300   
251   VEGN 2022-10-10  30.350000  30.350000  29.629999  29.830000    4500   
252   VEGN 2022-10-11  29.760000  29.860001  29.540001  29.540001    3200   

      adjusted  close_yesterday  dependent  
0    39.524410              NaN          0  
1    39.489620        39.759998          0  
2    39.687439        39.724998          0  
3    40.524448        39.924000          1  
4    40.878345        40.765999          0  
..         ...              ...        ...  
248  31.628000        31.719999          0  
249  31.309999        31.628000         -1  
250  30.239000        31.309999         -1  
251  29.830000        30.239000         -1  
252  29.540001        29.830000          0  

[253 rows x 10 columns]

In [222]:
basetable_dep = stock_information[['date', 'dependent']]
basetable_dep.head()

date  dependent
0 2021-10-11          0
1 2021-10-12          0
2 2021-10-13          0
3 2021-10-14          1
4 2021-10-15          0

## 5.3 Select Features and Group By Name

In [185]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- full_text: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- urls: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- display_url: string (nullable = true)
 |    |    |-- expanded_url: string (nullable = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- url: string (nullable = true)


Features:
- number_tweets
- favorite_count
- retweet_count
- followers_count
- emoji_count
- num_words
- num_hashtags
- num_mentions
- num_exclamation_marks
- engagement_rate
- sentiment_vader
- polarity
- subjectivity

In [190]:
number_of_tweets = df.groupBy('date').agg(F.countDistinct("full_text"))\
                    .withColumnRenamed("count(full_text)", "number_tweets") 
number_of_tweets.show()

+----------+-------------+
|      date|number_tweets|
+----------+-------------+
|2022-07-31|         3788|
|2022-03-28|         1952|
|2022-08-02|         8160|
|2022-06-22|         1060|
|2022-05-26|          434|
|2022-03-16|         1174|
|2022-07-07|         1050|
|2022-03-15|         1204|
|2022-03-12|          502|
|2022-08-20|          490|
|2022-08-11|          545|
|2022-05-01|          485|
|2022-01-09|          631|
|2022-01-15|          584|
|2022-04-27|         9026|
|2021-11-01|        20951|
|2022-09-07|         1013|
|2022-09-06|            4|
|2022-02-01|          684|
|2022-08-15|         1773|
+----------+-------------+
only showing top 20 rows



In [191]:
number_of_favorites = df.groupBy('date').agg(F.sum("favorite_count"))\
                        .withColumnRenamed("sum(favorite_count)", "number_of_favorites") 
number_of_favorites.show()

+----------+-------------------+
|      date|number_of_favorites|
+----------+-------------------+
|2022-07-31|              30457|
|2022-03-28|              18533|
|2022-08-02|              72537|
|2022-06-22|               9382|
|2022-05-26|               3965|
|2022-03-16|               8857|
|2022-07-07|               7177|
|2022-03-15|               8564|
|2022-03-12|               6269|
|2022-08-11|               1440|
|2022-08-20|               6142|
|2022-05-01|               9455|
|2022-01-09|               4773|
|2022-01-15|               4988|
|2022-04-27|              70083|
|2021-11-01|             159869|
|2022-09-06|                  5|
|2022-09-07|               6094|
|2022-02-01|               8620|
|2022-07-05|               7466|
+----------+-------------------+
only showing top 20 rows



In [192]:
number_of_retweets = df.groupBy('date').agg(F.sum("retweet_count"))\
                        .withColumnRenamed("sum(retweet_count)", "number_of_retweets") 
number_of_retweets.show()

+----------+------------------+
|      date|number_of_retweets|
+----------+------------------+
|2022-07-31|              3678|
|2022-03-28|              6313|
|2022-08-02|              8321|
|2022-06-22|              3070|
|2022-05-26|              1285|
|2022-03-16|              2973|
|2022-07-07|              2390|
|2022-03-15|              2426|
|2022-03-12|              2584|
|2022-08-11|               378|
|2022-08-20|              2001|
|2022-05-01|              1667|
|2022-01-09|              1454|
|2022-01-15|              1310|
|2022-04-27|             10022|
|2021-11-01|             26899|
|2022-09-06|                 1|
|2022-09-07|              1453|
|2022-02-01|              1224|
|2022-07-05|              2827|
+----------+------------------+
only showing top 20 rows



In [193]:
number_of_followers = df.groupBy('date').agg(F.sum("followers_count"))\
                        .withColumnRenamed("sum(followers_count)", "number_of_followers") 
number_of_followers.show()

+----------+-------------------+
|      date|number_of_followers|
+----------+-------------------+
|2022-07-31|           32478823|
|2022-03-28|           19476231|
|2022-08-02|           89026921|
|2022-06-22|            4919891|
|2022-05-26|            1658594|
|2022-03-16|            9743592|
|2022-07-07|            5866302|
|2022-03-15|            9292780|
|2022-03-12|            2403831|
|2022-08-11|            2114590|
|2022-08-20|           13507377|
|2022-05-01|            1792821|
|2022-01-09|            9529304|
|2022-01-15|            3845498|
|2022-04-27|           91430365|
|2021-11-01|          259167877|
|2022-09-06|               5837|
|2022-09-07|            6471303|
|2022-02-01|            4189796|
|2022-07-05|            4556302|
+----------+-------------------+
only showing top 20 rows



In [194]:
avg_emojis = df.groupBy('date').agg(F.avg("emoji_count"))\
                        .withColumnRenamed("avg(emoji_count)", "avg_emojis") 
avg_emojis.show()

+----------+-------------------+
|      date|         avg_emojis|
+----------+-------------------+
|2022-07-31|0.44051362683438156|
|2022-03-28| 0.8217213114754098|
|2022-08-02| 0.4500916310323763|
|2022-06-22| 0.6773584905660377|
|2022-05-26| 0.8387096774193549|
|2022-03-16| 0.7529812606473595|
|2022-07-07| 0.8533333333333334|
|2022-03-15| 0.8156146179401993|
|2022-03-12| 0.8784860557768924|
|2022-08-11| 0.7779816513761468|
|2022-08-20| 0.8306122448979592|
|2022-05-01| 0.8639175257731959|
|2022-01-09| 0.7448494453248812|
|2022-01-15|  1.018835616438356|
|2022-04-27| 0.4835639180962922|
|2021-11-01| 0.6724033393964833|
|2022-09-06|                0.0|
|2022-09-07| 0.8104639684106614|
|2022-02-01| 0.8771929824561403|
|2022-07-05| 0.6882168925964547|
+----------+-------------------+
only showing top 20 rows



In [195]:
avg_words = df.groupBy('date').agg(F.avg("num_words"))\
                        .withColumnRenamed("avg(num_words)", "avg_words") 
avg_words.show()

+----------+------------------+
|      date|         avg_words|
+----------+------------------+
|2022-07-31|24.518343815513628|
|2022-03-28| 24.45952868852459|
|2022-08-02|23.463042150274894|
|2022-06-22|  25.0311320754717|
|2022-05-26|25.179723502304146|
|2022-03-16|25.563032367972742|
|2022-07-07|             25.92|
|2022-03-15|24.631229235880397|
|2022-03-12| 26.40239043824701|
|2022-08-11| 26.34862385321101|
|2022-08-20|              25.3|
|2022-05-01| 23.51958762886598|
|2022-01-09|25.893819334389857|
|2022-01-15|26.241438356164384|
|2022-04-27|23.320088544548977|
|2021-11-01|24.398022480294763|
|2022-09-06|             22.25|
|2022-09-07|24.975320829220138|
|2022-02-01|25.263157894736842|
|2022-07-05| 25.08237747653806|
+----------+------------------+
only showing top 20 rows



In [196]:
avg_hashtags = df.groupBy('date').agg(F.avg("num_hashtags"))\
                        .withColumnRenamed("avg(num_hashtags)", "avg_hashtags") 
avg_hashtags.show()

+----------+------------------+
|      date|      avg_hashtags|
+----------+------------------+
|2022-07-31|1.2348008385744236|
|2022-03-28| 5.600409836065574|
|2022-08-02|1.0891875381795968|
|2022-06-22| 6.488679245283019|
|2022-05-26| 5.912442396313364|
|2022-03-16| 6.349233390119251|
|2022-07-07| 6.746666666666667|
|2022-03-15|  6.01578073089701|
|2022-03-12| 6.145418326693227|
|2022-08-11|  6.01651376146789|
|2022-08-20| 5.783673469387755|
|2022-05-01| 6.092783505154639|
|2022-01-09| 6.375594294770206|
|2022-01-15| 6.193493150684931|
|2022-04-27| 1.053901494189264|
|2021-11-01|1.3282496152231706|
|2022-09-06|              3.75|
|2022-09-07| 6.250740375123396|
|2022-02-01| 6.220760233918129|
|2022-07-05| 6.692387904066736|
+----------+------------------+
only showing top 20 rows



In [197]:
avg_num_mentions = df.groupBy('date').agg(F.avg("num_mentions"))\
                        .withColumnRenamed("avg(num_mentions)", "avg_num_mentions") 
avg_num_mentions.show()

+----------+-------------------+
|      date|   avg_num_mentions|
+----------+-------------------+
|2022-07-31|  0.995020964360587|
|2022-03-28|0.48360655737704916|
|2022-08-02| 0.8927306047648137|
|2022-06-22|0.45943396226415095|
|2022-05-26| 0.6820276497695853|
|2022-03-16|0.44122657580919933|
|2022-07-07| 0.4438095238095238|
|2022-03-15| 0.4642857142857143|
|2022-03-12| 0.5836653386454184|
|2022-08-11| 0.5394495412844037|
|2022-08-20|0.40408163265306124|
|2022-05-01|0.41649484536082476|
|2022-01-09| 0.5641838351822503|
|2022-01-15|  0.577054794520548|
|2022-04-27|  1.169120088544549|
|2021-11-01|  1.268364348677767|
|2022-09-06|               1.25|
|2022-09-07|  0.420533070088845|
|2022-02-01| 0.4605263157894737|
|2022-07-05|0.38686131386861317|
+----------+-------------------+
only showing top 20 rows



In [198]:
avg_exclamation_marks = df.groupBy('date').agg(F.avg("num_exclamation_marks"))\
                        .withColumnRenamed("avg(num_exclamation_marks)", "avg_exclamation_marks") 
avg_exclamation_marks.show()

+----------+---------------------+
|      date|avg_exclamation_marks|
+----------+---------------------+
|2022-07-31|  0.20335429769392033|
|2022-03-28|    0.367827868852459|
|2022-08-02|   0.2255345143555284|
|2022-06-22|   0.3849056603773585|
|2022-05-26|   0.4009216589861751|
|2022-03-16|   0.3475298126064736|
|2022-07-07|   0.4180952380952381|
|2022-03-15|   0.3795681063122924|
|2022-03-12|   0.3784860557768924|
|2022-08-11|   0.3779816513761468|
|2022-08-20|  0.29591836734693877|
|2022-05-01|  0.31752577319587627|
|2022-01-09|   0.3502377179080824|
|2022-01-15|   0.4246575342465753|
|2022-04-27|   0.2338682899833979|
|2021-11-01|   0.2843150972435987|
|2022-09-06|                 0.25|
|2022-09-07|   0.3889437314906219|
|2022-02-01|  0.41812865497076024|
|2022-07-05|  0.34515119916579773|
+----------+---------------------+
only showing top 20 rows



In [199]:
avg_engagement_rate = df.groupBy('date').agg(F.avg("engagement_rate"))\
                        .withColumnRenamed("avg(engagement_rate)", "avg_engagement_rate") 
avg_engagement_rate.show()

+----------+--------------------+
|      date| avg_engagement_rate|
+----------+--------------------+
|2022-07-31| 0.03940482673814084|
|2022-03-28|0.027124744099529717|
|2022-08-02| 0.03425039486468692|
|2022-06-22|0.026958683839033448|
|2022-05-26| 0.02479576533996367|
|2022-03-16| 0.02800456531026404|
|2022-07-07| 0.03767893216787877|
|2022-03-15|0.025491587779778387|
|2022-03-12|0.032361240852285506|
|2022-08-11|0.008771075731235242|
|2022-08-20|0.028147337167543075|
|2022-05-01|0.032178279526179174|
|2022-01-09| 0.01836276723013108|
|2022-01-15|0.030508797262298694|
|2022-04-27|0.053121477596350115|
|2021-11-01|0.035796467569625116|
|2022-09-06|5.323270054196484E-4|
|2022-09-07|0.044313609859705484|
|2022-02-01| 0.05057768184309863|
|2022-07-05|0.027553145419711638|
+----------+--------------------+
only showing top 20 rows



In [200]:
avg_sentiment_vader = df.groupBy('date').agg(F.avg("sentiment_vader"))\
                        .withColumnRenamed("avg(sentiment_vader)", "sentiment_vader") 
avg_sentiment_vader.show()

+----------+-------------------+
|      date|    sentiment_vader|
+----------+-------------------+
|2022-07-31|0.13780817610062898|
|2022-03-28|0.16072489754098357|
|2022-08-02|0.13525448992058645|
|2022-06-22|0.17772735849056595|
|2022-05-26|0.15763133640552993|
|2022-03-16|0.15785689948892667|
|2022-07-07|0.16888666666666677|
|2022-03-15|0.15796511627906978|
|2022-03-12|0.15765139442231077|
|2022-08-11|0.15588073394495408|
|2022-08-20|0.15766734693877565|
|2022-05-01|0.17410515463917528|
|2022-01-09| 0.1649524564183834|
|2022-01-15| 0.1764417808219178|
|2022-04-27| 0.1384954067515219|
|2021-11-01|0.14668564899025227|
|2022-09-06|              0.156|
|2022-09-07|0.15967028627838115|
|2022-02-01| 0.1637543859649122|
|2022-07-05|0.15292179353493224|
+----------+-------------------+
only showing top 20 rows



In [202]:
avg_polarity = df.groupBy('date').agg(F.avg("polarity"))\
                        .withColumnRenamed("avg(polarity)", "avg_polarity") 
avg_polarity.show()

+----------+-------------------+
|      date|       avg_polarity|
+----------+-------------------+
|2022-07-31|0.10821469260733596|
|2022-03-28| 0.1884670041360661|
|2022-08-02|0.12154470211661908|
|2022-06-22|0.21354227349158275|
|2022-05-26| 0.2113358760871926|
|2022-03-16|0.16750349272398937|
|2022-07-07|0.21019660344197633|
|2022-03-15|0.19116037771071578|
|2022-03-12|0.17529737841312965|
|2022-08-11|0.18246015815527605|
|2022-08-20| 0.1783665045682031|
|2022-05-01| 0.1964158457848812|
|2022-01-09|0.20437169155938453|
|2022-01-15|0.21327515936783936|
|2022-04-27|0.12120248559707147|
|2021-11-01|0.14879245544541778|
|2022-09-06| 0.3366161616161616|
|2022-09-07|0.17173510747532394|
|2022-02-01|0.19933466293829522|
|2022-07-05|0.18611731228741135|
+----------+-------------------+
only showing top 20 rows



In [203]:
avg_subjectivity = df.groupBy('date').agg(F.avg("subjectivity"))\
                        .withColumnRenamed("avg(subjectivity)", "avg_subjectivity") 
avg_subjectivity.show()

+----------+-------------------+
|      date|   avg_subjectivity|
+----------+-------------------+
|2022-07-31|0.40911586530328414|
|2022-03-28| 0.4198464149429641|
|2022-08-02| 0.4137001408511641|
|2022-06-22|0.42817074511168196|
|2022-05-26|0.41634818733646123|
|2022-03-16| 0.4012224654591205|
|2022-07-07|0.43063931001151085|
|2022-03-15|0.42308740290588975|
|2022-03-12| 0.4224295324674166|
|2022-08-11| 0.4137568749451123|
|2022-08-20|0.40342674553039726|
|2022-05-01|0.42877414921303014|
|2022-01-09|0.43466282528429423|
|2022-01-15|  0.459322670527722|
|2022-04-27| 0.4149151988682309|
|2021-11-01| 0.4245268991661895|
|2022-09-06| 0.4527777777777777|
|2022-09-07|0.42109212841197036|
|2022-02-01| 0.4248360371811769|
|2022-07-05| 0.4149568439805666|
+----------+-------------------+
only showing top 20 rows



In [208]:
# create basetable
basetable_ind = number_of_tweets.join(number_of_favorites, "date", how="inner") \
                    .join(number_of_retweets, "date", how="inner") \
                    .join(number_of_followers, "date", how="inner") \
                    .join(avg_emojis, "date", how="inner") \
                    .join(avg_words, "date", how="inner") \
                    .join(avg_hashtags, "date", how="inner") \
                    .join(avg_num_mentions, "date", how="inner") \
                    .join(avg_exclamation_marks, "date", how="inner") \
                    .join(avg_engagement_rate, "date", how="inner") \
                    .join(avg_sentiment_vader, "date", how="inner") \
                    .join(avg_polarity, "date", how="inner") \
                    .join(avg_subjectivity, "date", how="inner") 
basetable_ind_pd = basetable_ind.toPandas()

In [210]:
basetable_ind_pd.head()

date  number_tweets  number_of_favorites  number_of_retweets  \
0  2022-03-28           1952                18533                6313   
1  2022-07-31           3788                30457                3678   
2  2022-08-02           8160                72537                8321   
3  2022-05-26            434                 3965                1285   
4  2022-06-22           1060                 9382                3070   

   number_of_followers  avg_emojis  avg_words  avg_hashtags  avg_num_mentions  \
0             19476231    0.821721  24.459529      5.600410          0.483607   
1             32478823    0.440514  24.518344      1.234801          0.995021   
2             89026921    0.450092  23.463042      1.089188          0.892731   
3              1658594    0.838710  25.179724      5.912442          0.682028   
4              4919891    0.677358  25.031132      6.488679          0.459434   

   avg_exclamation_marks  avg_engagement_rate  polarity  avg_polarity  \
0               0.367828             0.027125  0.188467      0.188467   
1               0.203354             0.039405  0.108215      0.108215   
2               0.225535             0.034250  0.121545      0.121545   
3               0.400922             0.024796  0.211336      0.211336   
4               0.384906             0.026959  0.213542      0.213542   

   avg_subjectivity  
0          0.419846  
1          0.409116  
2          0.413700  
3          0.416348  
4          0.428171

In [213]:
type(basetable_ind_pd['date'][0])

datetime.date

In [224]:
basetable_ind_pd['date'] = pd.to_datetime(basetable_ind_pd['date'])
type(basetable_ind_pd['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [225]:
type(basetable_dep['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [226]:
basetable = basetable_ind_pd.merge(basetable_dep, how='inner', on= 'date')

In [228]:
# store basetable as a .parquet file
basetable.to_parquet("./../../data/basetable.parquet")

In [229]:
# export basetable as a .json file
basetable.to_json("./../../data/basetable.json", orient="records", force_ascii=False, lines=True)